<div style='color: #690027;' markdown="1">
    <h1>Detectie van stomata</h1> 
</div>

<div style='color: #690027;' markdown="1">
    <h2>Het netwerk</h2> 
</div>

Voer onderstaande codecel uit om van de methodes in deze notebook gebruik te kunnen maken.

In [ ]:
import imp
with open('../.scripts/diep_neuraal_netwerk.py', 'rb') as fp:
    diep_neuraal_netwerk = imp.load_module('.scripts', fp, '../.scripts/diep_neuraal_netwerk.py', ('.py', 'rb', imp.PY_SOURCE))
diep_neuraal_netwerk.laad_referentie_model()

Het ingeladen model is in staat om in afbeeldingen naar stomata te zoeken. Hoe dit gebeurt kan je vinden in [notebook 1](BasisDiepNeuraalNetwerk.ipynb) en [notebook 2](Overfitting.ipynb). Eerst kijken we hoe goed het model presteert op 3 vaste afbeeldingen. Je kan de threshold aanpassen met de schuifbalk.

In [ ]:
diep_neuraal_netwerk.vind_stomata()

Je kan ook je eigen prentje testen. Hou er wel rekening mee dat het model op zoek gaat naar 1 stoma in een gebied van 120 pixels op 120 pixels. Als de stomata op jouw afbeelding een andere verhouding hebben kan het zijn dat er geen stomata gevonden worden. 

Door volgende codecel uit te voeren kan je zelf een bestand van op jouw computer kiezen.

In [ ]:
display(diep_neuraal_netwerk.upload_widget)

Voer volgende codecel uit om de stomata te vinden. Dit kan enige tijd duren.

In [ ]:
diep_neuraal_netwerk.vind_stomata_eigen_afbeelding()

<div style='color: #690027;' markdown="1">
    <h2>Extra</h2> 
</div>

Het is ook mogelijk het netwerk te misleiden. Zo kan je zien dat het netwerk niet echt leert wat een stoma is maar gewoon patronen leert herkennen. Voer volgende codecel uit om te zien hoe goed het model doet met de foto van een kat (de ogen lijken op stomata).

In [ ]:
diep_neuraal_netwerk.misleid_netwerk()